In [1]:
import pyatmos
atmos = pyatmos.Simulation(docker_image="registry.gitlab.com/frontierdevelopmentlab/astrobiology/pyatmos",
                           DEBUG=True)
atmos.start()

Initializing Docker...
Pulling latest image... registry.gitlab.com/frontierdevelopmentlab/astrobiology/pyatmos
Initialization complete: 2018-07-31 17:06:54
Starting Docker container...
Container 'determined_lewin' running at 2018-07-31 17:06:55.


In [ ]:
atmos.run(species_concentrations={'O2' : 0.25}, 
          max_photochem_iterations=10000, 
          max_clima_steps=400, 
          output_directory='/Users/Will/Documents/FDL/results/run4'
         )

generic run ... cd /code/atmos
DEBUG run: cd /code/atmos
DEBUG _copy_container_file: docker cp determined_lewin:/code/atmos/PHOTOCHEM/INPUTFILES/species.dat /var/folders/t1/xhhxt12502bcgyn1r77j10m00000gn/T/tmp35w0raro
DEBUG _write_container_file: docker cp /var/folders/t1/xhhxt12502bcgyn1r77j10m00000gn/T/tmp27t0h_qh determined_lewin:/code/atmos/PHOTOCHEM/INPUTFILES/species.dat
Modified species file with:
{'O2': 0.25}
generic run ... cd /code/atmos && ./Photo.run
DEBUG _run_photochem: cd /code/atmos && ./Photo.run
DEBUG _copy_container_file: docker cp determined_lewin:/code/atmos/PHOTOCHEM/OUTPUT/out.out /var/folders/t1/xhhxt12502bcgyn1r77j10m00000gn/T/tmpkf7btoef
photochem finished after 40 iterations
DEBUG _run_photochem: photochem took 0 seconds
DEBUG _copy_container_file: docker cp determined_lewin:/code/atmos/PHOTOCHEM/OUTPUT/out.out /Users/Will/Documents/FDL/results/run4
DEBUG _copy_container_file: docker cp determined_lewin:/code/atmos/PHOTOCHEM/OUTPUT/out.dist /Users/Will/Docume

In [3]:
atmos.get_metadata()

{'atmos_start_time': 1533081625,
 'photochem_duration': 0,
 'photochem_iterations': 40,
 'clima_duration': 231,
 'atmos_run_duraton': 234,
 'input_max_clima_iterations': 400,
 'input_max_photochem_iterations': 10000,
 'input_species_concentrations': {'O2': 0.25}}

In [5]:
atmos.close()

Exiting...
Container killed.
